In [1]:
import pandas as pd
import numpy as np


In [2]:
from urllib.parse import urlparse
from datetime import timedelta

In [3]:
df_raw = pd.read_json('instapaper-export.json')

In [4]:
len(df_raw.index)

2153

In [5]:
', '.join(df_raw.columns)

'date_shift, preview, reading_time, starred, tag, title, url'

In [6]:
df_raw['hostname'] = df_raw.apply(lambda r: urlparse(r['url']).hostname, axis='columns')


In [7]:
def parse_date_shift(x):
    x = x.replace('ago', '').replace('Today', '1 days').replace('Yesterday', '2 days')
    x = x.strip()
    n, f = x.split()
    n = int(n)
    if 'day' in f:
        return n
    if 'week' in f:
        return n * 7
    if 'month' in f:
        return n * 31
    if 'year' in f:
        return n * 365
    return None

df_raw['days_ago'] = df_raw['date_shift'].apply(parse_date_shift)

In [8]:
def parse_read_time(x):
    """
    1 of 2 min remaining | 2 min
    """
    if x:
        if 'of' in x:
            return int(x.split()[2])
        return int(x.split()[0])
    return None

df_raw['read_time'] = df_raw['reading_time'].apply(parse_read_time)

Берем данные за последний год

In [9]:
df = df_raw[df_raw['days_ago'] < 366]

Смотрим количество постов

In [10]:
len(df.index)

1039

In [11]:
len(df[df['starred']].index)

288

In [12]:
df.groupby('tag')['url'].count()

tag
Favorites     39
Look           5
archive      995
Name: url, dtype: int64

Выбираем 16 наиболее читаемых сайтов

In [13]:
df.groupby(['hostname'])['url'].count().nlargest(16)


hostname
habrahabr.ru             315
geektimes.ru             123
baguzin.ru                42
github.com                37
medium.com                29
blog.jetbrains.com        25
www.cockroachlabs.com     24
www.yegor256.com          21
coreos.com                17
www.dpreview.com          16
aws.amazon.com            15
www.mirantis.com          15
www.youtube.com           13
ru.wikipedia.org          11
blog.rust-lang.org        10
eax.me                    10
Name: url, dtype: int64

Выбираем 16 наиболее читаемых сайтов из отмеченных постов

In [14]:
df[df['starred']].groupby(['hostname'])['url'].count().nlargest(16)

hostname
habrahabr.ru             81
geektimes.ru             39
baguzin.ru               17
medium.com               13
www.cockroachlabs.com    12
github.com               10
aws.amazon.com            6
blog.jetbrains.com        4
blog.rust-lang.org        4
dz.livejournal.com        4
www.mirantis.com          4
www.scylladb.com          4
www.yegor256.com          4
exonum.com                3
morepypy.blogspot.com     3
ru.wikipedia.org          3
Name: url, dtype: int64

Выбираем 8 наиболее читаемых сайтов из лучших постов

In [15]:
df[df['tag'] == 'Favorites'].groupby(['hostname'])['url'].count().nlargest(8)

hostname
habrahabr.ru             8
baguzin.ru               5
www.cockroachlabs.com    4
geektimes.ru             2
github.com               2
telegra.ph               2
antirez.com              1
apenwarr.ca              1
Name: url, dtype: int64

Суммарное время чтения всех постов

In [16]:
str(timedelta(minutes=df['read_time'].sum()))

'4 days, 11:46:00'

Статистика по времени чтения постов

In [17]:
df['read_time'].describe()

count    1039.000000
mean        6.223292
std         8.968485
min         0.000000
25%         2.000000
50%         4.000000
75%         7.000000
max        98.000000
Name: read_time, dtype: float64

Средние времся чтения статьи на 16 наиболее популярных сайтах

In [18]:
df.sort_values(by=['hostname'])\
    .groupby(['hostname'])['read_time']\
    .agg(['mean', 'count'])\
    .sort_values(['count'], ascending=False)\
    .head(16)

,mean,count
hostname,,
habrahabr.ru,6.888889,315
geektimes.ru,3.292683,123
baguzin.ru,20.190476,42
github.com,9.567568,37
medium.com,4.413793,29
blog.jetbrains.com,2.520000,25
www.cockroachlabs.com,5.083333,24
www.yegor256.com,2.666667,21
coreos.com,3.764706,17
